<a href="https://colab.research.google.com/github/boyerb/Investments/blob/master/Ex19-Fundamental_Law.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Investments: Theory, Fundamental Analysis, and Data Driven Analytics**, Bates, Boyer, and Fletcher

# Example Chapter 19: The Fundamental Law of Asset Management
In this example we provide a covariance matrix and vector of expected returns. The script then creates the efficinet fronter of risky securitities (EFRS), identifies the tangent portfolio, and creates a plot of both.  


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
